In [1]:
import sys
import os
sys.path = [p for p in sys.path if "ParaView" not in p]
sys.path.append("/home/haseeb/Notebooks/Parameteric_DMD_cylinder_2D_modules")



import torch as pt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as mtri
from flowtorch import DATASETS
from flowtorch.data import FOAMDataloader, mask_box
from pydmd import DMD, HODMD, ParametricDMD
from ezyrb import POD, Database, RBF, GPR, Linear
import seaborn as sns


from pathlib import Path
import re
# For High Quality Visuals
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 160  # crisp visuals


Refer to the installation instructions at https://github.com/FlowModelingControl/flowtorch
If you are not using the TecplotDataloader, ignore this warning.


In [ ]:
# Training and test parameters 

# Reynolds numbers
Re_list = np.array([100, 110, 120, 130, 140, 160, 170, 180, 190, 200])
Re_test = 150  # test parameter

# Plot
fig = plt.figure(figsize=(10, 2))
plt.title('Parameter Re')

# Plot all Re values as training
plt.plot(Re_list, np.zeros_like(Re_list), 'o', label='Train')

# Plot test Re separately
plt.plot(Re_test, 0, 'x', markersize=10, label='Test', color='red')

# Formatting
plt.xticks(np.append(Re_list, Re_test), rotation=30)
plt.yticks([])
plt.grid()
plt.legend(ncols=2)

plt.tight_layout()
plt.show()

In [ ]:
print(dir(ParametricDMD))


In [ ]:
# Load and prepare data

from data_loader import load_all_snapshots, load_test_parameter

# Base folder containing simulation data
base_path = os.path.expanduser("~/OpenFOAM/test_1")


# Time windows
training_window = (10.0, 15.0)  # Time range for training snapshots
future_window = (15.0, 20.0)    # Time range for future prediction
test_window = (3.0, 20.0)       # Time range for test Reynolds number

# Sampling step for downsampling time steps
sampling_step = 1

# Load training and future snapshots for all training Reynolds numbers
data = load_all_snapshots(
    Re_list=Re_list,
    base_path=base_path,
    mask_box=mask_box,
    FOAMDataloader=FOAMDataloader,
    training_window=training_window,
    future_window=future_window,
    sampling_step=sampling_step
)

# Access training data
snapshot_dict = data["snapshot_dict"]
sampled_times_dict = data["sampled_times_dict"]
snapshot_future_dict = data["snapshot_future_dict"]
sampled_times_future_dict = data["sampled_times_future_dict"]
masked_coords_dict = data["masked_coords_dict"]        
num_points_dict = data["num_points_dict"]              

# Load test snapshots for an unseen Reynolds number
path_test = f"{base_path}/cylinder_2D_Re{Re_test}"
snapshot_test, num_points_test, times_test, mask_test, coords_test, loader_test = load_test_parameter(
    Re=Re_test,
    path=path_test,
    mask_box=mask_box,
    FOAMDataloader=FOAMDataloader,
    time_window=test_window,
    sampling_step=sampling_step
)




In [ ]:
# Confirm shapes of loaded data

for Re in Re_list:
    print(f"Re={Re}, shape={snapshot_dict[Re].shape}")


## Preprocess training snapshots: Subtract mean flow and normalize the snapshots

In [ ]:
from preprocess_snapshots import preprocess_snapshots

# Preprocess training snapshots: subtract mean flow and normalize each block
train_snapshots, mean_flow, snapshot_processed_dict, norm_scales = preprocess_snapshots(
    snapshot_dict=snapshot_dict,
    Re_list=Re_list
)

# Shape of the mean flow vector
print("Mean flow shape:", mean_flow.shape)

# Shape of the stacked training snapshot array
print("Normalized training snapshot array shape:", train_snapshots.shape)

# Shape and normalization scale for each Reynolds number
for Re in Re_list:
    print(f"Re={Re}: processed shape = {snapshot_processed_dict[Re].shape}, norm scale = {norm_scales[Re]:.4f}")





In [ ]:
# Snapshot magnitude over time for each Parameter i.e., Reynolds number

from visualization_utilities import plot_snapshot_magnitudes

loader_dict = data["loader_dict"]  

plot_snapshot_magnitudes(
    snapshot_dict=snapshot_dict,
    sampled_times_dict=sampled_times_dict,
    Re_list=Re_list,
    loader_dict=loader_dict,  
    normalize_by_inlet=True
)


In [ ]:
# Plot POD cummulative energy residuals analysis for the training snapshots

from visualization_utilities import compute_pod, get_thresholds, plot_cumulative_energy, plot_residual_energy


# Compute POD energy and residuals
cumulative_energy, residual_content = compute_pod(snapshot_dict, Re_list)

tau_list = [1e-2, 1e-3, 1e-4, 1e-5]

# Get ranks for your specified tau thresholds
num_modes_99, tau_ranks = get_thresholds(residual_content, threshold=0.99, tau_list=tau_list)

# Plot cumulative energy with zoom
plot_cumulative_energy(cumulative_energy, threshold=0.99, num_modes_99=num_modes_99)

# Plot residual energy with colored summary
plot_residual_energy(residual_content, tau_ranks)


## Offline Phase

## Step 1: Perform POD and DMD on the concatenated snapshot matrix X1

In [ ]:
# POD and Parameteric DMD

# Step 1: Create shared POD basis
rom = POD(rank=30, method="randomized_svd")

# Step 2: Create DMD instances
trained_dmds = [DMD(svd_rank=-1) for _ in Re_list]

# Step 3: Create interpolator
interpolator = RBF()

# Step 4: Construct ParametricDMD
pdmd = ParametricDMD(trained_dmds, rom, interpolator)

# Step 6: Fit model
pdmd.fit(train_snapshots, np.array(Re_list).reshape(-1, 1))


In [ ]:
# Visualize eigenvalue spectrum of trained DMD models

plt.figure(figsize=(8, 6))

for i, dmd in enumerate(pdmd._dmd):
    eigs = dmd.eigs
    Re = Re_list[i]
    plt.scatter(np.real(eigs), np.imag(eigs), label=f"Re = {Re}", alpha=0.7)

plt.xlabel("Re(λ)")
plt.ylabel("Im(λ)")
plt.title("Eigenvalue Spectrum of Trained DMD Models")
plt.grid(True)
plt.axis("equal")
plt.legend(loc="best", fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# Plot comparison of DMD modal coefficients for a specific Reynolds number

from visualization_utilities import plot_dmd_modal_comparison


plot_dmd_modal_comparison(
    pdmd=pdmd,                             # Fitted ParametricDMD object containing trained DMD models
    Re_list=Re_list,                       # List of Reynolds numbers used during training
    sampled_times_dict=sampled_times_dict, # Dictionary mapping each Re to its physical time vector
    Re_value=160,                          # Reynolds number to inspect (must be in the parameter list)
    n_modes_to_plot=5                      # Number of dominant modes to visualize
)


In [ ]:
# Plot comparison of DMD FFT modal coefficients for a specific Reynolds number

from visualization_utilities import plot_dmd_fft_comparison

Re_target = 160 # Reynolds number to inspect

n_plot = 4  # Number of DMD modes to include in the FFT plots

dt = 0.01
plot_dmd_fft_comparison(pdmd, Re_list, Re_target, n_plot, dt)


In [ ]:
# Velocity field comparison between DMD prediction and true simulation during the training time window

from visualization_utilities import plot_flow_comparison_dmd_vs_true

# Set target Reynolds number and time range
Re_target = 200
t_start = 14.0
t_end = 15.0
granularity = 0.1

# Plot flow comparison between DMD prediction and true simulation
plot_flow_comparison_dmd_vs_true(
    Re_target=Re_target,
    Re_list=Re_list,
    t_start=t_start,
    t_end=t_end,
    granularity=granularity,
    rom=rom,
    pdmd=pdmd,
    mean_flow=mean_flow,
    norm_scales=norm_scales,
    sampled_times_dict=sampled_times_dict,
    snapshot_processed_dict=snapshot_processed_dict,
    masked_coords_dict=masked_coords_dict,
    num_points_dict=num_points_dict,
    cmap='icefire'  
)



In [ ]:
# 🔒 Cache trained DMD internals before interpolation
cached_dmd_list = pdmd._dmd.copy()  # List of trained DMD instances
cached_modal_coeffs = pdmd.training_modal_coefficients.copy()  # List of modal coefficient arrays



# Online phase

# Forecasting and interpolation for unknown Re

In [ ]:
# Define forecasting time range

# Use sampled times from the first Re in your training list
training_times = np.array(sampled_times_dict[Re_list[0]], dtype=float)

pdmd.dmd_time["t0"]   = pdmd.original_time["tend"] 
pdmd.dmd_time["tend"] = pdmd.original_time["tend"] + 500 - 1e-12
pdmd.dmd_time["dt"]   = pdmd.original_time["dt"]


# Set target Re for interpolation
pdmd.parameters = np.array([[150]])


# Trigger reconstruction (includes forecasting)
forecasted_snapshots = pdmd.reconstructed_data  # shape: (1, space_dim, time_steps)
forecasted_field = forecasted_snapshots[0]

# Print physical time steps
print(
    f"Forecasting from t = {pdmd.dmd_time['t0']} to t = {pdmd.dmd_time['tend']} with Δt = {pdmd.dmd_time['dt']}"
)
print("Time vector:", pdmd.dmd_timesteps)



In [ ]:
# Plot modal coefficient comparison Interpolated vs True

from visualization_utilities import plot_dmd_modal_comparison_interp_vs_true

# Number of DMD modes to visualize
n_modes_to_plot = 6

plot_dmd_modal_comparison_interp_vs_true(
    pdmd=pdmd,
    snapshot_test=snapshot_test,
    loader_test=loader_test,
    mean_flow=mean_flow,
    Re_test=Re_test,
    dt_phys=0.01,
    t0_phys=15.0,
    n_modes_to_plot=n_modes_to_plot,
    match_tolerance=0.05
)



In [ ]:
# Plot FFT comparison of interpolated modal coefficients vs true

from visualization_utilities import plot_fft_modal_comparison_interp_vs_true


plot_fft_modal_comparison_interp_vs_true(
    pdmd=pdmd,
    snapshot_test=snapshot_test,
    loader_test=loader_test,
    mean_flow=mean_flow,
    Re_test=Re_test,
    dt_phys=0.01,
    t0_phys=15.0,
    n_modes_to_plot=6,
    match_tolerance=0.05
)

In [ ]:
# Plot velocity field comparison between interpolated DMD forecast and true simulation for the test Reynolds number
from visualization_utilities import plot_flow_comparison_interpolated_dmd_vs_true

t_start = 18.0
t_end = 19.0
granularity = 0.1
norm_scale = norm_scales[160]
plot_flow_comparison_interpolated_dmd_vs_true(
    pdmd=pdmd,
    snapshot_test=snapshot_test,
    sampled_times_test=times_test,
    loader_test=loader_test,  # This is the class, not an instance — only needed if your function uses it
    mask_test=mask_test,
    num_points_test=num_points_test,
    norm_scale=norm_scale,
    mean_flow=mean_flow,
    Re_test=Re_test,
    t_start=t_start,
    t_end=t_end,
    granularity=granularity,
    cmap="icefire"
)

